In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from copy import copy
pd.options.plotting.backend = 'plotly'
from plotly import graph_objects as go
from aust_covid.inputs import load_raw_pop_data
from general_utils.tex_utils import StandardTexDoc
PROJECT_PATH = Path().resolve().parent

In [ ]:
def get_ifrs(tex_doc):
    description = 'Age-specific infection fatality rates (IFRs) were estimated by various groups ' \
        "in unvaccinated populations, including O'Driscoll and colleagues who estimated " \
        'IFRs using data from 45 countries. These IFRs pertained to the risk of death given infection ' \
        'for the wild-type strain of SARS-CoV-2 in unvaccinated populations, and so are unlikely to represent ' \
        'IFRs that would be applicable to the Australian population in 2022 because of vaccine-induced immunity ' \
        'and differences in severity for the variants we simulated. ' \
        'We therefore considered more recent studies, such as that of Erikstrup and colleagues to be better ' \
        'applicable to our local context, although also with limitations. ' \
        '*** Insert brief description of Erikstrup study here. ***' \
        "As expected, the estimates from Erikstrup are consideraly lower than those of O'Driscoll. " \
        'However, there are also several potential differences between the Danish epidemic and that of Australia, ' \
        'most notably that community transmission had been established from much earlier in the pandemic in ' \
        'Denmark than in Australia. Further, given that these estimates estimate attack rates from blood donors, ' \
        'the age ranges covered by this study extend from 17 years to 73 years of age, making it necessary ' \
        'to extrapolate from these estimates to the extremes of age. ' \
        'We approached this extrapolation by identifying broadly equivalent younger and older age groups ' \
        'from each study for use as baselines for the more extreme age groups. Specifically, ' \
        'we considered that the IFR estimate for the 17 to 36 years-old age group from Erikstrup could ' \
        "be compared to the 25 to 29 years-old age group form O'Driscoll, and that " \
        'the 61 to 73 years-old age group from Erikstrup could be compared to the 65 to 69 years-old ' \
        "age group from O'Driscoll. We next calculated the ratio in the IFRs of these `equivalent' " \
        'age groups from each study, before then applying these ratios to the estimates from ' \
        "O'Driscoll for the age bands outside of the age range calculated by Erikstrup. " \
        '(i.e. 0-4, 5-9, 10-14, 15-19, 70-74, 75-79 and 80+). ' \
        'Next, to obtain IFR estimates for each modelled 5-year band from 75-79 years-old ' \
        'we performed linear interpolation from the estimates available to the mid-point of each modelled age band. ' \
        'We now have estimates for each 5-year band from 0-4 to 75-79 and for 80+ years-old. ' \
        'To calculate the IFR parameter for the modelled 75+ age band, we took an average of the 75-79 and 80+ ' \
        'estimates, weighted using the proportion of the Australian population aged 75+ who are aged ' \
        '75-79 and 80+. '
    tex_doc.add_line(description, 'Parameters', subsection='Infection fatality rates')
    
    # Raw data from O'Driscoll, 5-year age bands
    odriscoll = pd.Series(
        {
            0: 0.003,
            5: 0.001,
            10: 0.001,
            15: 0.003,
            20: 0.006,
            25: 0.013,
            30: 0.024,
            35: 0.04,
            40: 0.075,
            45: 0.121,
            50: 0.207,
            55: 0.323,
            60: 0.456,
            65: 1.075,
            70: 1.674,
            75: 3.203,
            80: 8.292,
        }
    ) / 100.0
    odriscoll.index = odriscoll.index + 2.5

    # Raw data from Erikstrup
    erikstrup = pd.Series(
        {
            (17 + 36) / 2: 2.6,
            (36 + 51) / 2: 5.8,
            (51 + 61) / 2: 14.6,
            (61 + 73) / 2: 24.6,
        }
    ) / 1e5

    # Most comparable upper and lower age group ratio
    lower_ratio = erikstrup[26.5] / odriscoll[27.5]
    upper_ratio = erikstrup[67.0] / odriscoll[67.5]

    # Apply the ratios to the upper and missing age groups without estimates from Erikstrup
    lower_adjusted = odriscoll[ :17.5] * lower_ratio
    upper_adjusted = odriscoll[72.5: ] * upper_ratio

    # Combine extrapolated estimates with Erikstrup
    combined = pd.concat([lower_adjusted, erikstrup, upper_adjusted])

    # Modelled breakpoints (lower values rather than midpoints of age groups)
    age_mid_points = np.linspace(2.5, 72.5, 15)
    final_values = pd.Series(np.interp(age_mid_points, combined.index, combined), index=age_mid_points)

    # Proportion of the 75+ age group IFR to take from the 80+ estimate
    pops = load_raw_pop_data('31010do002_202206.xlsx').sum(axis=1)
    prop_75_over_80 = pops['80-84': ].sum() / pops['75-79': ].sum()
    final_values[77.5] = combined[82.5] * prop_75_over_80 + combined[77.5] * (1.0 - prop_75_over_80)

    # Set age bands back to lower breakpoint values
    model_breakpoint_values = copy(final_values)
    model_breakpoint_values.index = final_values.index - 2.5
    
    return odriscoll, erikstrup, upper_adjusted, lower_adjusted, combined, final_values, model_breakpoint_values

In [ ]:
app_doc = StandardTexDoc(PROJECT_PATH / 'supplement', 'supplement', "Australia's 2023 Omicron Waves Supplement", 'austcovid')
odriscoll, erikstrup, upper_adjusted, lower_adjusted, combined, final_values, model_breakpoint_values = get_ifrs(app_doc)
scaling_factor = 0.04
fig = go.Figure()
fig.add_trace(go.Scatter(x=odriscoll.index, y=odriscoll, name="O'Driscoll"))
fig.add_trace(go.Scatter(x=erikstrup.index, y=erikstrup, name='Erikstrup'))
fig.add_trace(go.Scatter(x=upper_adjusted.index, y=upper_adjusted, name="Upper adjusted O'Driscoll"))
fig.add_trace(go.Scatter(x=lower_adjusted.index, y=lower_adjusted, name="Lower adjusted O'Driscoll"))
fig.add_trace(go.Scatter(x=combined.index, y=combined, name="Combined Erikstrup/O'Driscoll"))
fig.add_trace(go.Scatter(x=final_values.index, y=final_values, name='Combined and interpolated'))
fig.add_trace(go.Scatter(x=model_breakpoint_values.index, y=model_breakpoint_values, name='Values by model breakpoints'))
fig.update_yaxes(type='log')